In [ ]:
import sys
import os
import warnings

# Ajouter le répertoire parent pour inclure 'src'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src')))

# Imports après avoir ajusté le sys.path
from src.backtester.back_tester import BackTesting
from src.strategies.strategies import *
from src.utils.utilities import Utilities
from src.performance.graph import IndexPlotter
import utils.constant as constant
from src.performance.metrics import MetricsCalculator

# Supprimer les avertissements FutureWarning et RuntimeWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option('display.float_format', '{:.3f}'.format)

<div style="background-color: rgb(15, 50, 110); color: white; padding: 15px; border-radius: 8px; box-shadow: 0 3px 6px 0 rgba(0, 0, 0, 0.2); text-align: center;">
    <h1 style="font-size: 32px; font-weight: bold; margin: 0;">Volatility Timing under Low-Volatility Strategy</h1>
</div> 

## <span style="color: rgb(15, 50, 150); font-size: 20px; font-weight: bold;"> Volatility Strategies </span>

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;"> 1. Strategies construction and back-testing </span>

In [ ]:
# Paramètres du back-test
params = {
    "currency": constant.CURRENCY,
    "start_date": constant.START_DATE,
    "end_date": constant.END_DATE,
    "ticker": constant.TICKER,
    "strategy": VolatilityTimingStrategy(),
    "use_pickle_universe": constant.USE_PICKLE_UNIVERSE,
}

# Lancement du back-test
asset_index = BackTesting.start(params)
#Utilities.save_data_to_pickle(asset_index, file_name="VolatilityTiming",folder_subpath="asset_indices\\monthly_eq_weighted")

In [ ]:
# Appel de la fonction port
asset_index = Utilities.get_data_from_pickle('VolatilityTiming',folder_subpath="asset_indices\\monthly_eq_weighted")
asset_index.get_port_file('VolatilityTiming')

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;"> 2. Volatility and Return Profile under Different Market Conditions </span>

In [ ]:
asset_indices_monthly = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile", "HighVolatilityDecile"], 
                                             folder_subpath="asset_indices\\monthly_eq_weighted")
other_data = Utilities.get_data_from_pickle('other_US_data')
IndexPlotter.asset_indices_plot_under_diff_conditions(asset_indices_monthly,other_data)

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;"> 3. Performance and Wealth Plots of the Volatility Strategies </span>

In [ ]:
# Graph des tracks records des stratégies
asset_indices_monthly = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile", "HighVolatilityDecile", 
                                              "VolatilityTiming", "VolatilityTiming2sided"],
                                             folder_subpath="asset_indices\\monthly_eq_weighted")
other_data = Utilities.get_data_from_pickle('other_US_data')
IndexPlotter.plot_track_records(asset_indices_monthly, other_data['USRINDEX Index'])

In [ ]:
IndexPlotter.asset_indices_barplot(asset_indices_monthly, other_data)

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  4. Performance and Risk Measures of the Various Volatility Portfolios </span>

In [ ]:
IndexPlotter.display_asset_indices_metrics(asset_indices_monthly,other_data)

## <span style="color: rgb(15, 50, 150); font-size: 20px; font-weight: bold;"> Robustness Tests </span>


### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  1. CAPM and Fama–French Alphas </span>

In [ ]:
CPAM_stats, FF_stats = IndexPlotter.display_regress_statistics(asset_indices_monthly)
CPAM_stats

In [ ]:
FF_stats

### <span style="color: rgb(15, 50, 150); font-size: 16px; font-weight: bold;">  2. Performance plots for various rebalancing frequencies </span>

In [ ]:
# Chargement des indices
asset_indices_quaterly = Utilities.load_asset_indices(["LowVolatilityDecile", "MidVolatilityDecile", "HighVolatilityDecile", 
                                              "VolatilityTiming", "VolatilityTiming2sided"],
                                             folder_subpath="asset_indices\\quarterly_eq_weighted")
asset_indices_semiannually = Utilities.load_asset_indices(["VolatilityTiming"],
                                             folder_subpath="asset_indices\\semiannually_eq_weighted")
asset_indices_annually = Utilities.load_asset_indices(["VolatilityTiming"],
                                             folder_subpath="asset_indices\\annually_eq_weighted")

In [ ]:
# Graph des tracks records des stratégies : rebalancement trimestriel (fin trimestre)
IndexPlotter.plot_track_records(asset_indices_quaterly, other_data['USRINDEX Index'])

In [ ]:
# Graphique des tracks records de la stratégie VolTiming pour différentes fréquences de rebalancement
IndexPlotter.plot_tracks_for_diff_rebalncing_freq( [asset_indices_monthly['VolatilityTiming'],
                                                   asset_indices_quaterly['VolatilityTiming'],
                                                   asset_indices_semiannually['VolatilityTiming'],
                                                   asset_indices_annually['VolatilityTiming']],
                                                   label_names=["monthly", "quarterly","semiannually","annually"],
                                                   strat_name="VolatilityTiming")